In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
cd ..

/Users/amiyaguchi/wikipedia-retention


# Spark Preprocessing

In [4]:
from src.data.snap_import_user_projection import UnimodalUserProjection
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.getOrCreate()

input_path = "data/processed/enwiki-meta-compact"
model = UnimodalUserProjection(spark).extract(input_path).transform()

In [5]:
import shutil
import glob
import os

def write_csv(df, name, overwrite=False):
    interim_path = "data/interim/{}".format(name)
    processed_file = "data/processed/{}.csv".format(name)
    if os.path.exists(processed_file) and not overwrite:
        print("file already exists")
        return
    df.repartition(1).write.csv(interim_path, header=True, mode="overwrite")
    interim_file = glob.glob("{}/*.csv".format(interim_path))[0]
    shutil.copy(interim_file, processed_file)
    shutil.rmtree(interim_path)
    print("wrote file to {}".format(processed_file))

In [6]:
spark.table("bipartite").cache()
spark.table("bipartite").count()

53595946

In [7]:
projection = spark.sql("""
select
    t1.article_id as e1,
    t2.article_id as e2,
    t1.user_id
from bipartite t1
join bipartite t2
on t1.user_id = t2.user_id
where t1.article_id < t2.article_id
    -- and t1.edit_date = t2.edit_date
""")

%time projection.count()

Py4JJavaError: An error occurred while calling o31.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 172 in stage 7.0 failed 1 times, most recent failure: Lost task 172.0 in stage 7.0 (TID 839, localhost, executor driver): ExecutorLostFailure (executor driver exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 7208823 ms
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1887)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1875)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1874)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1874)
	at org.apache.spark.schedu